In [26]:
import requests
import datetime
import sqlite3 as lite
import pandas as pd
import matplotlib.pyplot as plt
import collections
import scipy.stats as stats
import numpy as np



In [2]:
cities = {"Austin": '30.303936,-97.754355', "Boston": '42.331960,-71.020173', "New_York": '40.663619,-73.938589', "Philadelphia": '40.009376,-75.133346', "Seattle": '47.620499,-122.350876'}

In [3]:
APIKEY = 'dc6b130f33e13a20201deb99ea5d59c1'

In [4]:
#connect to db
con = lite.connect('weather.db')
cur = con.cursor()

In [5]:
#From text - get start date and iterate until the present day
start_date = datetime.datetime.now()-datetime.timedelta(days=30)

In [6]:
start_date = datetime.datetime.now()-datetime.timedelta(days=30)
start_date = start_date.replace(minute=0, hour=0, second=0, microsecond=0)
total_start_date = start_date

In [7]:
#https://api.forecast.io/forecast/APIKEY/LATITUDE,LONGITUDE,TIME
url = 'https://api.forecast.io/forecast/' + APIKEY

In [86]:
#Example 
#Austin = 'https://api.forecast.io/forecast/dc6b130f33e13a20201deb99ea5d59c1/30.303936,-97.754355'

In [87]:
#Test
#r = requests.get("https://api.forecast.io/forecast/dc6b130f33e13a20201deb99ea5d59c1/30.303936,-97.754355")

In [8]:
r.status_code

NameError: name 'r' is not defined

In [15]:
r.json().keys()

[u'hourly',
 u'currently',
 u'longitude',
 u'flags',
 u'daily',
 u'offset',
 u'latitude',
 u'timezone',
 u'minutely']

In [11]:
#query = "DROP TABLE daily_temp"
#cur.execute(query)

OperationalError: no such table: daily_temp

In [14]:
#Create table
with con:
    cur.execute('CREATE TABLE daily_temp ( day INT, Austin REAL, Boston REAL, New_York REAL, Philadelphia REAL, Seattle REAL)')

In [13]:
cur.execute(query)

In [15]:
with con:
    while start_date<datetime.datetime.now():
        cur.execute('INSERT INTO daily_temp (day) VALUES (?)', (int(start_date.strftime('%s')),)) #inserting the time in Unix time 
        start_date += datetime.timedelta(days=1)

In [16]:
for k, v in cities.iteritems():
    start_date = total_start_date
    while start_date < datetime.datetime.now():
        #query for the value
        r = requests.get(url + '/' + v + ',' + start_date.strftime('%Y-%m-%dT12:00:00')) # insert url+lat/long+time
        #insert temperatureMax into the db
        with con:
            query = 'UPDATE daily_temp SET ' + k + '=' + str(r.json()['daily']['data'][0]['temperatureMax']) + ' WHERE day = ' + start_date.strftime('%s')
            print query
            cur.execute(query)
        
        #increment query_date to the next day for next operation of loop
        start_date += datetime.timedelta(days=1) #increment query_date to the next day

con.close() # a good practice to close connection to database

UPDATE daily_temp SET New_York=77.81 WHERE day = 1432962000
UPDATE daily_temp SET New_York=78.03 WHERE day = 1433048400
UPDATE daily_temp SET New_York=60.37 WHERE day = 1433134800
UPDATE daily_temp SET New_York=54.83 WHERE day = 1433221200
UPDATE daily_temp SET New_York=63.27 WHERE day = 1433307600
UPDATE daily_temp SET New_York=63.34 WHERE day = 1433394000
UPDATE daily_temp SET New_York=63.06 WHERE day = 1433480400
UPDATE daily_temp SET New_York=74.94 WHERE day = 1433566800
UPDATE daily_temp SET New_York=67.92 WHERE day = 1433653200
UPDATE daily_temp SET New_York=73.69 WHERE day = 1433739600
UPDATE daily_temp SET New_York=81.38 WHERE day = 1433826000
UPDATE daily_temp SET New_York=77.51 WHERE day = 1433912400
UPDATE daily_temp SET New_York=87.06 WHERE day = 1433998800
UPDATE daily_temp SET New_York=82.7 WHERE day = 1434085200
UPDATE daily_temp SET New_York=82.7 WHERE day = 1434171600
UPDATE daily_temp SET New_York=80.18 WHERE day = 1434258000
UPDATE daily_temp SET New_York=78.49 WHERE

In [66]:
#Fixing No JSON error - added '/' to request
r = url + '/' + v + ',' + (start_date.strftime('%Y-%m-%dT12:00:00'))

In [67]:
r

'https://api.forecast.io/forecast/dc6b130f33e13a20201deb99ea5d59c1/40.663619,-73.938589,2015-05-16T12:00:00'

In [17]:
con = lite.connect('weather.db')
cur = con.cursor()

In [18]:
df = pd.read_sql_query("SELECT * FROM daily_temp ORDER BY day",con,index_col='day')

In [76]:
df

,Austin,Boston,New_York,Philadelphia,Seattle
day,,,,,
1432357200,84.65,63.25,66.20,71.07,60.03
1432443600,80.95,80.50,73.86,81.79,60.77
1432530000,81.39,78.12,81.01,87.04,59.77
1432616400,88.14,83.62,80.62,88.53,64.88
1432702800,84.22,82.63,75.45,88.39,70.30
1432789200,86.05,83.03,79.01,85.37,75.41
1432875600,88.51,70.50,78.62,87.84,71.85
1432962000,85.50,84.30,77.81,87.02,67.36
1433048400,84.99,75.84,78.03,90.55,68.29


In [ ]:
#What's the range of temperatures for each city? 
#Whatis the mean temperature for each city? 
#What's the variance? Are there any patterns in the data? 
#Which cities had the largest temperature changes over the time period?

In [19]:
#What's the range of temperatures for each city? 
max(df['Seattle'])-min(df['Seattle'])
a = max(df['Austin'])-min(df['Austin'])
b = max(df['Boston'])-min(df['Boston'])
c = max(df['New_York'])-min(df['New_York'])
d = max(df['Philadelphia'])-min(df['Philadelphia'])
e = max(df['Seattle'])-min(df['Seattle'])
print a, b, c, d, e

13.38 37.46 32.61 34.79 26.41


In [20]:
#Whatis the mean temperature for each city? 
a = df['Austin'].mean()
b = df['Boston'].mean()
c = df['New_York'].mean()
d = df['Philadelphia'].mean()
e = df['Seattle'].mean()
print a, b, c, d, e

88.2412903226 70.3674193548 75.4758064516 81.2451612903 73.4858064516


In [21]:
#What's the variance? 
a = df['Austin'].var()
b = df['Boston'].var()
c = df['New_York'].var()
d = df['Philadelphia'].var()
e = df['Seattle'].var()
print a, b, c, d, e

10.9541316129 97.3120664516 66.4319051613 69.5010924731 46.118671828


In [ ]:
#Just as you did with the Citi Bike data, go through the the sequence 
#of temperature readings and calculate the change between days and
#find the greatest range in high temperatures in the month you measured. 
#Which city had the greatest variation? 
#What is the distribution of the difference? Does the result surprise you? 
#Why or why not?

In [23]:
daily_change = collections.defaultdict(int) #defaultdict should contain integer values
for col in df.columns: # loop through columns in dataframe
    city_vals = df[col].tolist() #creates list of values?
    city_id = col 
    city_change = 0
    for k, v in enumerate(city_vals): 
        if k < len(city_vals) - 1:
            city_change += abs(city_vals[k] - city_vals[k+1])
    daily_change[str(city_id)] = city_change #city_id is str and city_change int is added

In [24]:
daily_change

defaultdict(<type 'int'>, {'New_York': 161.06, 'Boston': 178.59999999999991, 'Seattle': 111.65000000000001, 'Austin': 60.130000000000052, 'Philadelphia': 168.61999999999995})

In [22]:
#Greatest variation = Boston
a = df['Austin'].std()
b = df['Boston'].std()
c = df['New_York'].std()
d = df['Philadelphia'].std()
e = df['Seattle'].std()
print a, b, c, d, e

3.3097026472 9.86468785373 8.15057698334 8.33673152219 6.79107295116


In [37]:
#Test normal distribution
plt.figure()
test_data = df['Boston']   
graph1 = stats.probplot(test_data, dist="norm", plot=plt)
plt.show()

In [45]:
plt.hist(boston_vals, histtype='bar')
plt.show()

In [43]:
boston_vals = df['Boston'].tolist()